In [1]:
# Importing the libraries

from typing import Optional, Tuple, List

import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
"cuda" if torch.cuda.is_available() else "cpu"

'cuda'

In [2]:
torch.manual_seed(1337)

In [3]:
# Loading and printing the first 1000 characters of the text

with open("../data/tinyshakespeare.txt", "r") as f:
    text = f.read()

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [9]:
# Extracting the vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [10]:
# Creating a dictionary to map the characters to integers (tokenizers)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

def encode(text: str) -> List[int]:
    return [stoi[ch] for ch in text]

def decode(text: List[int]) -> str:
    return "".join([itos[ch] for ch in text])

print(encode("Hello, students!"))
print(decode(encode("Hello, students!")))

[20, 43, 50, 50, 53, 6, 1, 57, 58, 59, 42, 43, 52, 58, 57, 2]
Hello, students!


In [11]:
# Encoding part of the text

data = torch.tensor(encode(text[:1000]), dtype=torch.int64)
data

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [32]:
# Encoding the whole text
data = torch.tensor(encode(text), dtype=torch.int64)

In [22]:
# Splitting the data into training and validation sets
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [27]:
# Checking input to the model + 1
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [30]:
# Printing the input and target sequences

x = train_data[:block_size]
y = train_data[1 : block_size + 1]

for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when input is {context}, the target is {target}")

when input is tensor([18]), the target is 47
when input is tensor([18, 47]), the target is 56
when input is tensor([18, 47, 56]), the target is 57
when input is tensor([18, 47, 56, 57]), the target is 58
when input is tensor([18, 47, 56, 57, 58]), the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [32]:
# Contructing the batches

batch_size = 4
block_size = 8


def get_batch(split: str) -> Tuple[torch.Tensor, torch.Tensor]:
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("---------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()}, the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[54, 43, 39, 49,  8,  0,  0, 18],
        [46, 43, 63,  1, 58, 46, 47, 52],
        [ 0,  0, 18, 47, 56, 57, 58,  1],
        [45, 53, 53, 42,  1, 41, 47, 58]])
targets:
torch.Size([4, 8])
tensor([[43, 39, 49,  8,  0,  0, 18, 47],
        [43, 63,  1, 58, 46, 47, 52, 49],
        [ 0, 18, 47, 56, 57, 58,  1, 15],
        [53, 53, 42,  1, 41, 47, 58, 47]])
---------
when input is [54], the target is 43
when input is [54, 43], the target is 39
when input is [54, 43, 39], the target is 49
when input is [54, 43, 39, 49], the target is 8
when input is [54, 43, 39, 49, 8], the target is 0
when input is [54, 43, 39, 49, 8, 0], the target is 0
when input is [54, 43, 39, 49, 8, 0, 0], the target is 18
when input is [54, 43, 39, 49, 8, 0, 0, 18], the target is 47
when input is [46], the target is 43
when input is [46, 43], the target is 63
when input is [46, 43, 63], the target is 1
when input is [46, 43, 63, 1], the target is 58
when input is [46, 43, 63, 1, 5

## Bigram part

In [56]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size: int) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(
        self, idx: torch.Tensor, targets: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        logits = self.token_embedding_table(
            idx
        )  # (batch_size, block_size, vocab_size) -> (B,T,C)

        # If targets is None, we are in inference mode
        if targets is None:
            return logits, None

        B, T, C = logits.shape
        logits = logits.view(B * T, C)  # (batch_size * block_size, vocab_size)
        targets = targets.view(-1)  # (batch_size * block_size)

        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: torch.Tensor, max_new_tokens: int) -> torch.Tensor:
        """
        Generates new tokens given a sequence of tokens.

        Args:
            idx: (batch_size, block_size) tensor containing the tokens
            max_new_tokens: maximum number of tokens to generate

        Returns:
            (batch_size, block_size + max_new_tokens) tensor containing the original tokens and the generated tokens
        """
        for _ in range(max_new_tokens):
            logits, _ = self(idx)

            logits = logits[:, -1, :]  # (batch_size, vocab_size)
            probs = F.softmax(logits, dim=-1)  # (batch_size, vocab_size)

            idx_next = torch.multinomial(probs, 1)
            idx = torch.cat([idx, idx_next], dim=1)

        return idx

In [65]:
# Checking out the model without training

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.int64)
print(decode(m.generate(idx, max_new_tokens=10)[0].tolist()))

torch.Size([256, 65])
tensor(4.5100, grad_fn=<NllLossBackward0>)

llfIJ
uaAb


In [66]:
# Configuring the optimizer

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [67]:
# Defining the training loop
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.023599624633789


In [68]:
idx = torch.zeros((1, 1), dtype=torch.int64)
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))


Fit wecir uis d Cis, led lve t pars t wn ustheshe ow'l, l penXzear
Noulafayitird.
bellved t ars tis antild plus ansondave itho
Fitieaunghe'lve ce paice Mansuthe t.

Fiecushabun:
Fizes: urestilvex



Wes Mactoussules thise wen rstopey sorthit an: atine
Wer, ir:

S; d ito metht iusurf t u k wen't ictqffourthey; hacceve ghirs bjesus't.
Le:
so am, ren Cincous m, uiknmHx3Poonne rcthum, reranerizeakit ons Citit, Cizer as ey, ary!



Wery, ut oris r:
Wed s kin:
Bemichen:
wn:
Ald orspl:
ispa or thaw'the


## Baby steps to Attention

In [35]:
# Creating toy example

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
print(x)
x.shape

tensor([[[-0.8345,  0.5978],
         [-0.0514, -0.0646],
         [-0.4970,  0.4658],
         [-0.2573, -1.0673],
         [ 2.0089, -0.5370],
         [ 0.2228,  0.6971],
         [-1.4267,  0.9059],
         [ 0.1446,  0.2280]],

        [[ 2.4900, -1.2237],
         [ 1.0107,  0.5560],
         [-1.5935, -1.2706],
         [ 0.6903, -0.1961],
         [ 0.3449, -0.3419],
         [ 0.4759, -0.7663],
         [-0.4190, -0.4370],
         [-1.0012, -0.4094]],

        [[-1.6669, -1.3651],
         [-0.1655,  0.9623],
         [ 0.0315, -0.7419],
         [-0.2978,  0.0172],
         [-0.1772, -0.1334],
         [ 0.2940,  1.3850],
         [ 0.1209,  2.5418],
         [-0.6405, -1.9740]],

        [[-0.3296,  0.0080],
         [ 0.9262, -1.8846],
         [ 0.1670,  0.4586],
         [-1.7662,  0.5860],
         [ 1.7510,  0.2807],
         [ 0.3110, -0.6538],
         [-0.6576,  0.3184],
         [-0.5496, -1.4649]]])


torch.Size([4, 8, 2])

In [57]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = xprev.mean(dim=0) # (C)

In [45]:
x[0]

tensor([[-0.8345,  0.5978],
        [-0.0514, -0.0646],
        [-0.4970,  0.4658],
        [-0.2573, -1.0673],
        [ 2.0089, -0.5370],
        [ 0.2228,  0.6971],
        [-1.4267,  0.9059],
        [ 0.1446,  0.2280]])

In [46]:
xbow[0]

tensor([[-0.8345,  0.5978],
        [-0.4429,  0.2666],
        [-0.4610,  0.3330],
        [-0.4100, -0.0171],
        [ 0.0738, -0.1210],
        [ 0.0986,  0.0153],
        [-0.1193,  0.1425],
        [-0.0863,  0.1532]])

In [48]:
# Slightly better way to do it
torch.manual_seed(42)

a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2), dtype=torch.float32)
c = a @ b
print("a=")
print(a, a.shape)
print("------")
print("b=")
print(b, b.shape)
print("------")
print("c=")
print(c, c.shape)


a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]) torch.Size([3, 3])
------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) torch.Size([3, 2])
------
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]]) torch.Size([3, 2])


In [49]:
# Triangular matrix
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [50]:
# Slightly better way to do it
torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2), dtype=torch.float32)
c = a @ b
print("a=")
print(a, a.shape)
print("------")
print("b=")
print(b, b.shape)
print("------")
print("c=")
print(c, c.shape)


a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]]) torch.Size([3, 3])
------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) torch.Size([3, 2])
------
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]]) torch.Size([3, 2])


In [54]:
# sum([1, 2, 3, 4, 5]) / n = 1/n + 2/n + 3/n + 4/n + 5/n

torch.manual_seed(42)

a = torch.tril(torch.ones(3, 3))
a /= a.sum(dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2), dtype=torch.float32)
c = a @ b
print("a=")
print(a, a.shape)
print("------")
print("b=")
print(b, b.shape)
print("------")
print("c=")
print(c, c.shape)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]]) torch.Size([3, 3])
------
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]]) torch.Size([3, 2])
------
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]]) torch.Size([3, 2])


In [55]:
# Making it even better
wei = torch.tril(torch.ones(T, T))
wei /= wei.sum(dim=1, keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [56]:
wei = torch.tril(torch.ones((T, T)))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x
xbow2[0]

tensor([[-0.8345,  0.5978],
        [-0.4429,  0.2666],
        [-0.4610,  0.3330],
        [-0.4100, -0.0171],
        [ 0.0738, -0.1210],
        [ 0.0986,  0.0153],
        [-0.1193,  0.1425],
        [-0.0863,  0.1532]])

In [ ]:
# Per   gatve perbego miauksedamas juodas gyvunas su uodega  -    
# 0.125 0.125 0.125   0.125  0.3   0.125   0.3    0.125

In [73]:
# the most optimal version using softmax

tril = torch.tril(torch.ones((T, T)))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [74]:
xbow2

tensor([[[-0.8345,  0.5978],
         [-0.4429,  0.2666],
         [-0.4610,  0.3330],
         [-0.4100, -0.0171],
         [ 0.0738, -0.1210],
         [ 0.0986,  0.0153],
         [-0.1193,  0.1425],
         [-0.0863,  0.1532]],

        [[ 2.4900, -1.2237],
         [ 1.7503, -0.3338],
         [ 0.6357, -0.6461],
         [ 0.6494, -0.5336],
         [ 0.5885, -0.4953],
         [ 0.5697, -0.5404],
         [ 0.4285, -0.5257],
         [ 0.2498, -0.5111]],

        [[-1.6669, -1.3651],
         [-0.9162, -0.2014],
         [-0.6003, -0.3816],
         [-0.5247, -0.2819],
         [-0.4552, -0.2522],
         [-0.3303,  0.0206],
         [-0.2659,  0.3808],
         [-0.3127,  0.0865]],

        [[-0.3296,  0.0080],
         [ 0.2983, -0.9383],
         [ 0.2545, -0.4727],
         [-0.2506, -0.2080],
         [ 0.1497, -0.1103],
         [ 0.1766, -0.2008],
         [ 0.0574, -0.1267],
         [-0.0185, -0.2939]]])

In [75]:
xbow3

tensor([[[-0.8345,  0.5978],
         [-0.4429,  0.2666],
         [-0.4610,  0.3330],
         [-0.4100, -0.0171],
         [ 0.0738, -0.1210],
         [ 0.0986,  0.0153],
         [-0.1193,  0.1425],
         [-0.0863,  0.1532]],

        [[ 2.4900, -1.2237],
         [ 1.7503, -0.3338],
         [ 0.6357, -0.6461],
         [ 0.6494, -0.5336],
         [ 0.5885, -0.4953],
         [ 0.5697, -0.5404],
         [ 0.4285, -0.5257],
         [ 0.2498, -0.5111]],

        [[-1.6669, -1.3651],
         [-0.9162, -0.2014],
         [-0.6003, -0.3816],
         [-0.5247, -0.2819],
         [-0.4552, -0.2522],
         [-0.3303,  0.0206],
         [-0.2659,  0.3808],
         [-0.3127,  0.0865]],

        [[-0.3296,  0.0080],
         [ 0.2983, -0.9383],
         [ 0.2545, -0.4727],
         [-0.2506, -0.2080],
         [ 0.1497, -0.1103],
         [ 0.1766, -0.2008],
         [ 0.0574, -0.1267],
         [-0.0185, -0.2939]]])

## The Self-Attention part
### IMPORTANT
![](../attention.png)

In [ ]:
# Per gatve perbego miauksedamas juodas gyvunas su uodega kate
# Queries for these tokens are "kate", "gyvunas", "uodega"
# Keys the these tokens are "gyvunas, kuris miauksi", "gyvas organizmas, kuris nera zmogus", "gyvuno kuno dalis, kuri yra vizginama"

In [109]:
torch.manual_seed(42)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16

key = nn.Linear(C, head_size)
query = nn.Linear(C, head_size)
value = nn.Linear(C, head_size)

k = key(x)
q = query(x)

wei = q @ k.transpose(-2, -1) # (B, T, head_size) @ (B, head_size, T) -> (B, T, T)

tril = torch.tril(torch.ones((T, T)))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v

In [111]:
out[0].shape

torch.Size([8, 16])

In [121]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * (head_size ** -0.5)

In [122]:
k.var()

tensor(1.1770)

In [123]:
q.var()

tensor(1.0797)

In [124]:
wei.var()

tensor(1.0607)